In [279]:
import torch
import torchvision
import lightly.models as models
import lightly.loss as loss
import lightly.data as data
import pytorch_lightning as pl
import torch.nn.functional as F
import math
import os
import shutil
from tqdm import tqdm
import pandas as pd
# for plotting
import os
from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.offsetbox as osb
from matplotlib import rcParams as rcp

from sklearn.metrics.pairwise import cosine_similarity

# for resizing images to thumbnails
import torchvision.transforms.functional as functional

# for clustering and 2d representations
from sklearn import random_projection
os.environ["CUDA_VISIBLE_DEVICES"]="1,0"

## Seperate train and test set

In [4]:
# for logo2k and logo targetlist
# train_list = [x.strip('\n') for x in open('/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/train_targets.txt').readlines()]
# test_list = [x.strip('\n') for x in open('/home/l/liny/ruofan/PhishIntention/src/siamese_retrain/test_targets.txt').readlines()]

# os.makedirs('../datasets/targetlist/train', exist_ok=True)
# os.makedirs('../datasets/targetlist/test', exist_ok=True)

# for data in train_list:
#     if not os.path.exists(os.path.join('../datasets/targetlist/train/', data.split('/')[0])):
#         os.makedirs('../datasets/targetlist/train/' + data.split('/')[0], exist_ok=True)
#     try:
#         shutil.copyfile(os.path.join('/home/l/liny/ruofan/PhishIntention/src/phishpedia/expand_targetlist/', data),
#                         os.path.join('../datasets/targetlist/train/', data))
#     except FileExistsError:
#         continue

# for data in test_list:
#     if not os.path.exists(os.path.join('../datasets/targetlist/test/', data.split('/')[0])):
#         os.makedirs('../datasets/targetlist/test/' + data.split('/')[0], exist_ok=True)
#     try:
#         shutil.copyfile(os.path.join('/home/l/liny/ruofan/PhishIntention/src/phishpedia/expand_targetlist/', data),
#                         os.path.join('../datasets/targetlist/test/', data))
#     except FileExistsError:
#         continue

In [21]:
# # for celebA
# train_list = [x.split(' ')[0] for x in open('../datasets/celebA/list_eval_partition.txt').readlines() \
#                       if x.strip().split(' ')[1] == '0']

# valid_list = [x.split(' ')[0] for x in open('../datasets/celebA/list_eval_partition.txt').readlines() \
#                       if x.strip().split(' ')[1] == '1']

# test_list = [x.split(' ')[0] for x in open('../datasets/celebA/list_eval_partition.txt').readlines() \
#                       if x.strip().split(' ')[1] == '2']


# os.makedirs('../datasets/celebA/train', exist_ok=True)
# os.makedirs('../datasets/celebA/valid', exist_ok=True)
# os.makedirs('../datasets/celebA/test', exist_ok=True)

# for data in train_list:
#     try:
#         shutil.copyfile(os.path.join('../datasets/celebA/img_align_celeba/img_align_celeba/', data),
#                         os.path.join('../datasets/celebA/train/', data))
#     except FileExistsError:
#         continue

# for data in valid_list:
#     try:
#         shutil.copyfile(os.path.join('../datasets/celebA/img_align_celeba/img_align_celeba/', data),
#                         os.path.join('../datasets/celebA/valid/', data))
#     except FileExistsError:
#         continue

# for data in test_list:
#     try:
#         shutil.copyfile(os.path.join('../datasets/celebA/img_align_celeba/img_align_celeba/', data),
#                         os.path.join('../datasets/celebA/test/', data))
#     except FileExistsError:
#         continue

In [140]:
# # CIFAR100
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [11]:
# traindict = unpickle('../datasets/cifar-100-python/train')

# os.makedirs('../datasets/CIFAR100/train', exist_ok=True)
# os.makedirs('../datasets/CIFAR100/test', exist_ok=True)

# for j, data in enumerate(traindict[b'data']):
#     filename = traindict[b'filenames'][j].decode('utf-8')
#     from PIL import Image
#     im = Image.fromarray(np.transpose(np.reshape(data, (3,32,32)), (1,2,0)))
#     im.save(os.path.join('../datasets/CIFAR100/train', filename))

In [10]:
# len(set(traindict[b'fine_labels']))

In [62]:
# testdict = unpickle('../datasets/cifar-100-python/test')
# for j, data in enumerate(testdict[b'data']):
#     filename = testdict[b'filenames'][j].decode('utf-8')
#     from PIL import Image
#     im = Image.fromarray(np.transpose(np.reshape(data, (3,32,32)), (1,2,0)))
#     im.save(os.path.join('../datasets/CIFAR100/test', filename))

In [64]:
# len(os.listdir('../datasets/CIFAR100/test/'))

In [24]:
# # clothing
# import pandas as pd
# df = pd.read_csv('../datasets/clothing-dataset/images.csv')
# train_imgs = list(df['image'])[:5000]
# test_imgs = list(df['image'])[5000:]

In [36]:
# os.makedirs('../datasets/clothing-dataset/train', exist_ok=True)
# os.makedirs('../datasets/clothing-dataset/test', exist_ok=True)

# for data in train_imgs:
#     try:
#         shutil.copyfile(os.path.join('../datasets/clothing-dataset/images', data+'.jpg'),
#                         os.path.join('../datasets/clothing-dataset/train', data+'.jpg'))
#     except FileExistsError:
#         continue
#     except FileNotFoundError:
#         continue

# for data in test_imgs:
#     try:
#         shutil.copyfile(os.path.join('../datasets/clothing-dataset/images', data+'.jpg'),
#                         os.path.join('../datasets/clothing-dataset/test', data+'.jpg'))
#     except FileExistsError:
#         continue
#     except FileNotFoundError:
#         continue

## Create dataloader, model

In [288]:
# the collate function applies random transforms to the input images
collate_fn = data.ImageCollateFunction(input_size=32, cj_prob=0.5)

# create a dataset from your image folder
dataset = data.LightlyDataset(input_dir='../datasets/CIFAR100/train/')

# build a PyTorch dataloader
train_dataloader = torch.utils.data.DataLoader(
    dataset,                # pass the dataset to the dataloader
    batch_size=128,         # a large batch size helps with the learning
    shuffle=True,           # shuffling is important!
    collate_fn=collate_fn,
    drop_last=False)  # apply transformations to the input images


# create a dataset from your image folder
dataset = data.LightlyDataset(input_dir='../datasets/CIFAR100/test/')

# build a PyTorch dataloader
test_dataloader = torch.utils.data.DataLoader(
    dataset,                # pass the dataset to the dataloader
    batch_size=128,         # a large batch size helps with the learning
    shuffle=True,           # shuffling is important!
    collate_fn=collate_fn,
    drop_last=False)  # apply transformations to the input images

In [289]:
# use a resnet50 backbone
resnet = torchvision.models.resnet.resnet50()
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])

# build the simsiam model
model = models.SimSiam(resnet, num_ftrs=2048)


## Get embeddings

In [290]:
model = torch.nn.DataParallel(model)

In [291]:
model.load_state_dict(torch.load('../output/CIFAR100.pt', map_location='cpu'))

<All keys matched successfully>

In [292]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

DataParallel(
  (module): SimSiam(
    (backbone): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
     

In [293]:
embeddings = []
filenames = []

# disable gradients for faster calculations
model.eval()
with torch.no_grad():
    for i, (x, _, fnames) in tqdm(enumerate(test_dataloader)):
        # move the images to the gpu
        x = x[0].to(device)
        # embed the images with the pre-trained backbone
        y = model.module.backbone(x)
        y = y.squeeze()
        y = F.normalize(y, dim=1)
        # store the embeddings and filenames in lists
        embeddings.append(y)
        filenames = filenames + list(fnames)

# concatenate the embeddings and convert to numpy
embeddings = torch.cat(embeddings, dim=0)
embeddings = embeddings.cpu()

79it [00:26,  2.96it/s]


In [294]:
embeddings.shape

torch.Size([9999, 2048])

In [ ]:
train_embeddings = []
train_filenames = []

# disable gradients for faster calculations
model.eval()
with torch.no_grad():
    for i, (x, _, fnames) in tqdm(enumerate(train_dataloader)):
        # move the images to the gpu
        x = x[0].to(device)
        # embed the images with the pre-trained backbone
        y = model.module.backbone(x)
        y = y.squeeze()
        y = F.normalize(y, dim=1)
        # store the embeddings and filenames in lists
        train_embeddings.append(y)
        train_filenames = train_filenames + list(fnames)

# concatenate the embeddings and convert to numpy
train_embeddings = torch.cat(train_embeddings, dim=0)
train_embeddings = train_embeddings.cpu()

283it [02:14,  2.70it/s]

In [ ]:
train_embeddings.shape

In [ ]:
len(train_filenames)

In [ ]:
def knn_predict(feature: torch.Tensor,
                feature_bank: torch.Tensor,
                feature_labels: torch.Tensor, 
                num_classes: int,
                knn_k: int=200,
                knn_t: float=0.1) -> torch.Tensor:
    """Run kNN predictions on features based on a feature bank
    This method is commonly used to monitor performance of self-supervised
    learning methods.
    The default parameters are the ones
    used in https://arxiv.org/pdf/1805.01978v1.pdf.
    Args:
        feature: 
            Tensor of shape [N, D] for which you want predictions
        feature_bank: 
            Tensor of a database of features used for kNN
        feature_labels: 
            Labels for the features in our feature_bank
        num_classes: 
            Number of classes (e.g. `10` for CIFAR-10)
        knn_k: 
            Number of k neighbors used for kNN
        knn_t: 
            Temperature parameter to reweights similarities for kNN
    Returns:
        A tensor containing the kNN predictions
    Examples:
        >>> images, targets, _ = batch
        >>> feature = backbone(images).squeeze()
        >>> # we recommend to normalize the features
        >>> feature = F.normalize(feature, dim=1)
        >>> pred_labels = knn_predict(
        >>>     feature,
        >>>     feature_bank,
        >>>     targets_bank,
        >>>     num_classes=10,
        >>> )
    """

    # compute cos similarity between each feature vector and feature bank ---> [B, N]
    sim_matrix = torch.mm(feature, feature_bank)
    # [B, K]
    sim_weight, sim_indices = sim_matrix.topk(k=knn_k, dim=-1)
    # [B, K]
    sim_labels = torch.gather(feature_labels.expand(feature.size(0), -1), dim=-1, index=sim_indices)
    
    # we do a reweighting of the similarities
    sim_weight = (sim_weight / knn_t).exp()
    
    # counts for each class
    one_hot_label = torch.zeros(feature.size(0) * knn_k, num_classes, device=sim_labels.device)
    
    # [B*K, C]
    one_hot_label = one_hot_label.scatter(dim=-1, index=sim_labels.view(-1, 1), value=1.0)
    # weighted score ---> [B, C]
    pred_scores = torch.sum(one_hot_label.view(feature.size(
        0), -1, num_classes) * sim_weight.unsqueeze(dim=-1), dim=1)
    pred_labels = pred_scores.argsort(dim=-1, descending=True)
    return pred_labels

In [262]:
# get labels for feature_bank and labels for test feature
# df = pd.read_csv('../datasets/clothing-dataset/images.csv')
# bank_labels = []
# feature_labels = []

# for file in filenames:
#     label = list(df.loc[df['image'] == file.split('.jpg')[0]]['label'])[0]
#     feature_labels.append(label)
    
# for file in train_filenames:
#     label = list(df.loc[df['image'] == file.split('.jpg')[0]]['label'])[0]
#     bank_labels.append(label)

In [ ]:
testdict = unpickle('../datasets/cifar100/test')
feature_labels = []
for file in tqdm(filenames):
    index = [x.decode('utf-8') for x in testdict[b'filenames']].index(file)
    cls = testdict[b'fine_labels'][index]
    feature_labels.append(cls)

traindict = unpickle('../datasets/cifar100/train')
bank_labels = []
for file in tqdm(train_filenames):
    index = [x.decode('utf-8') for x in traindict[b'filenames']].index(file)
    cls = traindict[b'fine_labels'][index]
    bank_labels.append(cls)

In [270]:
labeldict = {x:i for i, x in enumerate(set(feature_labels).union(set(bank_labels)))}

feature_label_int = [labeldict[x] for x in feature_labels]
bank_label_int = [labeldict[x] for x in bank_labels]

In [273]:
feature_label_int = torch.tensor(feature_label_int)
bank_label_int = torch.tensor(bank_label_int)

In [285]:
pred_labels = knn_predict(feature= embeddings,
                          feature_bank= train_embeddings.t().contiguous(),
                          feature_labels= bank_label_int, 
                          num_classes=len(labeldict.keys()))

In [296]:
num = len(embeddings)
top1 = (pred_labels[:, 0] == feature_label_int).float().sum()
print(top1/num)

tensor(0.0045)


In [ ]:
# clothing dataset (4995 training, 403 testing) Test NN Acc = 0.0045
# celebA dataset Test NN Acc = 
# CIFAR100 dataset (49999 training, 9999 testing) Test NN Acc = 
# Logo2k dataset Test NN Acc = 
# Logo targetlist dataset Test NN Acc = 
